In [94]:
import pandas as pd
import numpy as np

In [95]:
# loading both the csv files
credit = pd.read_csv('recommender_system/tmdb_5000_credits.csv')
movies = pd.read_csv('recommender_system/tmdb_5000_movies.csv')


## merging and selecting useful features for my project
movies = movies.merge(how = 'inner' , left_on='title' , right_on= 'title' , right= credit)
removing_col = ['budget' , 'homepage' , 'original_language' , 'original_title' , 'popularity' , 'production_companies' , 'production_countries' , 'revenue' , 'runtime' , 'spoken_languages' , 'tagline' , 'vote_average' , 'release_date' , 'status' , 'vote_count']
movies = movies.drop(removing_col , axis = 1)

In [96]:
movies.dropna(inplace= True)
movies.sample(5)

,genres,id,keywords,overview,title,movie_id,cast,crew
2998,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",9355,"[{""id"": 1395, ""name"": ""arena""}, {""id"": 1965, ""...",Mad Max becomes a pawn in a decadent oasis of ...,Mad Max Beyond Thunderdome,9355,"[{""cast_id"": 1, ""character"": ""Mad Max Rockatan...","[{""credit_id"": ""52fe44ecc3a36847f80b218b"", ""de..."
3827,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",8618,"[{""id"": 237, ""name"": ""gay""}, {""id"": 1412, ""nam...","The story of an unruly class of bright, funny ...",The History Boys,8618,"[{""cast_id"": 1, ""character"": ""Crowther"", ""cred...","[{""credit_id"": ""52fe44b0c3a36847f80a48d1"", ""de..."
1145,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 14, ""n...",10610,"[{""id"": 703, ""name"": ""detective""}, {""id"": 1456...",A Hong Kong detective suffers a fatal accident...,The Medallion,10610,"[{""cast_id"": 1, ""character"": ""Eddie Yang"", ""cr...","[{""credit_id"": ""52fe43939251416c75015bd7"", ""de..."
1111,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",834,"[{""id"": 810, ""name"": ""budapest""}, {""id"": 2849,...",As the war between the vampires and the Lycans...,Underworld: Evolution,834,"[{""cast_id"": 1, ""character"": ""Selene"", ""credit...","[{""credit_id"": ""52fe427ec3a36847f8023033"", ""de..."
3227,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",152747,"[{""id"": 3342, ""name"": ""yacht""}, {""id"": 41599, ...","Deep into a solo voyage in the Indian Ocean, a...",All Is Lost,152747,"[{""cast_id"": 7, ""character"": ""Our Man"", ""credi...","[{""credit_id"": ""548dab3f92514122ea00461e"", ""de..."


In [97]:
print(movies.shape)
credit.shape

(4806, 8)


(4803, 4)

## extracting the tags from the important columns

In [98]:
def mine(obj):
    '''this function is to mine the necessary stuff from the list of dictionaries in some important columns of the dataset'''
    obj = eval(obj)
    core = []
    for i in range(len(obj)):
        core.append(obj[i]['name'])
        
    return core

def mine_2(obj):
    '''for cast column'''
    obj = eval(obj)
    core_2 = []
    try:
        for i in range(3):
            core_2.append(obj[i]['name'])
            
    except:
        try:
            for i in range(2):
                core_2.append(obj[i]['name'])
        except:
            try:
                for i in range(1):
                    core_2.append(obj[i]['name'])
            except:
                pass
        
    return core_2

def mine_3(obj):
    '''for crew column as i want to get the name of only with job vlaue director'''
    obj = eval(obj)
    core_3 = []
    a = 0
    try:
        while True:
            if obj[a]['job'] == 'Director':
                core_3.append(obj[a]['name'])
            a +=1
    except:
        pass
    
    return core_3

movies['genres'] = movies['genres'].apply(mine)
movies['keywords'] = movies['keywords'].apply(mine)
movies['cast'] = movies['cast'].apply(mine_2)
movies['crew'] = movies['crew'].apply(mine_3)
movies['overview'] = movies['overview'].apply(lambda x : x.split())
movies['genres'] = movies['genres'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['cast'] = movies['cast'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['crew'] = movies['crew'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


## getting and working with the new dataframe 'df'

In [99]:
new_df = movies.loc[:,['movie_id' , 'title' , 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: (' '.join(x)).lower())

In [100]:
new_df.head(5)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


## feature extraction

In [101]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize , sent_tokenize
from nltk.stem import PorterStemmer

In [102]:
ps = PorterStemmer()
def stemming(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return ' '.join(y)

new_df['tags'] = new_df['tags'].apply(stemming)

cv = CountVectorizer(stop_words= 'english' , max_features= 4000)
vectors = cv.fit_transform(new_df['tags']).toarray()

In [103]:
from sklearn.metrics.pairwise import cosine_similarity
cos_theta = cosine_similarity(vectors)

In [106]:
def recommend(movie):
    index = (new_df[new_df['title'] == movie].index)
    type(index)
    similar_5_index = list(np.argsort(cos_theta[index]))[0][::-1][1:6]
    rec_movies = [new_df['title'][ind] for ind in similar_5_index]
    return rec_movies

recommend('Avengers: Age of Ultron')

['Iron Man 3', 'Iron Man 2', 'Iron Man', 'Thor', 'The Avengers']